In [1]:
import numpy as np
import os

In [19]:
GPU_NUM = 1
CPU_NUM = 6
JOB_TIME = '0-04:00'
JOB_MEMORY = '32000M'
PROCESS_SCRIPT = './draw_interest_area_on_videos.py'
job_sub_dir = './job_scripts'
job_out_dir = './job_scripts_output'
if not os.path.exists(job_sub_dir):
    os.makedirs(job_sub_dir)
if not os.path.exists(job_out_dir):
    os.makedirs(job_out_dir)

In [22]:
tasks = ['RoboschoolAnt-v1', 'RoboschoolHalfCheetah-v1', 'RoboschoolWalker2d-v1', 'RoboschoolHopper-v1']
dropout_rates = ['0', '0.01', '0.05', '0.1', '0.2']
for task in tasks:
    for dropout in dropout_rates:
        job_filename = 'job_{}_{}.sh'.format(task, dropout)
        print(job_filename)
        with open(os.path.join(job_sub_dir, job_filename), 'w') as job_file:
            job_file.write('#!/bin/bash\n')
            job_file.write('#SBATCH --account=def-dkulic\n')
            job_file.write('#SBATCH --gres=gpu:{}        # request GPU "generic resource"\n'.format(GPU_NUM))
            job_file.write('#SBATCH --cpus-per-task={}    #Maximum of CPU cores per GPU request: 6 on Cedar, 16 on Graham.\n'.format(CPU_NUM))
            job_file.write('#SBATCH --mem={}               # memory per node\n'.format(JOB_MEMORY))
            job_file.write('#SBATCH --time={}            # time (DD-HH:MM)\n'.format(JOB_TIME))
            job_file.write('#SBATCH --output=./job_scripts_output/dbedpg_{0}_{1}_%N-%j.out        # %N for node name, %j for jobID\n'.format(task, dropout))
            job_file.write('## Main processing command\n')
            job_file.write('module load cuda cudnn\n')
            job_file.write('source ~/tf_gpu/bin/activate\n')
            job_file.write('python ./ddpg_dropout.py  --env {0} --new_mlp --dropout_rate {1} --exp_name dbedpg_{0}_{1}'.format(task, dropout))


job_RoboschoolAnt-v1_0.sh
job_RoboschoolAnt-v1_0.01.sh
job_RoboschoolAnt-v1_0.05.sh
job_RoboschoolAnt-v1_0.1.sh
job_RoboschoolAnt-v1_0.2.sh
job_RoboschoolHalfCheetah-v1_0.sh
job_RoboschoolHalfCheetah-v1_0.01.sh
job_RoboschoolHalfCheetah-v1_0.05.sh
job_RoboschoolHalfCheetah-v1_0.1.sh
job_RoboschoolHalfCheetah-v1_0.2.sh
job_RoboschoolWalker2d-v1_0.sh
job_RoboschoolWalker2d-v1_0.01.sh
job_RoboschoolWalker2d-v1_0.05.sh
job_RoboschoolWalker2d-v1_0.1.sh
job_RoboschoolWalker2d-v1_0.2.sh
job_RoboschoolHopper-v1_0.sh
job_RoboschoolHopper-v1_0.01.sh
job_RoboschoolHopper-v1_0.05.sh
job_RoboschoolHopper-v1_0.1.sh
job_RoboschoolHopper-v1_0.2.sh


In [21]:
import os
job_sub_dir = './job_scripts'
jobs = os.listdir('./job_scripts')
jobs.sort()

for job in jobs:
    code = os.system('sbatch {}'.format(os.path.join(job_sub_dir, job)))
    print('{}: {}'.format(job, code))


./job_scripts/job_RoboschoolAnt-v1_0.01.sh
./job_scripts/job_RoboschoolAnt-v1_0.05.sh
./job_scripts/job_RoboschoolAnt-v1_0.1.sh
./job_scripts/job_RoboschoolAnt-v1_0.2.sh
./job_scripts/job_RoboschoolAnt-v1_0.sh
./job_scripts/job_RoboschoolHalfCheetah-v1_0.01.sh
./job_scripts/job_RoboschoolHalfCheetah-v1_0.05.sh
./job_scripts/job_RoboschoolHalfCheetah-v1_0.1.sh
./job_scripts/job_RoboschoolHalfCheetah-v1_0.2.sh
./job_scripts/job_RoboschoolHalfCheetah-v1_0.sh
./job_scripts/job_RoboschoolHopper-v1_0.01.sh
./job_scripts/job_RoboschoolHopper-v1_0.05.sh
./job_scripts/job_RoboschoolHopper-v1_0.1.sh
./job_scripts/job_RoboschoolHopper-v1_0.2.sh
./job_scripts/job_RoboschoolHopper-v1_0.sh
./job_scripts/job_RoboschoolWalker2d-v1_0.01.sh
./job_scripts/job_RoboschoolWalker2d-v1_0.05.sh
./job_scripts/job_RoboschoolWalker2d-v1_0.1.sh
./job_scripts/job_RoboschoolWalker2d-v1_0.2.sh
./job_scripts/job_RoboschoolWalker2d-v1_0.sh


In [8]:
job_sub_dir




['job_RoboschoolAnt-v1_0.01.sh',
 'job_RoboschoolAnt-v1_0.05.sh',
 'job_RoboschoolAnt-v1_0.1.sh',
 'job_RoboschoolAnt-v1_0.2.sh',
 'job_RoboschoolAnt-v1_0.sh',
 'job_RoboschoolHalfCheetah-v1_0.01.sh',
 'job_RoboschoolHalfCheetah-v1_0.05.sh',
 'job_RoboschoolHalfCheetah-v1_0.1.sh',
 'job_RoboschoolHalfCheetah-v1_0.2.sh',
 'job_RoboschoolHalfCheetah-v1_0.sh',
 'job_RoboschoolHopper-v1_0.01.sh',
 'job_RoboschoolHopper-v1_0.05.sh',
 'job_RoboschoolHopper-v1_0.1.sh',
 'job_RoboschoolHopper-v1_0.2.sh',
 'job_RoboschoolHopper-v1_0.sh',
 'job_RoboschoolWalker2d-v1_0.01.sh',
 'job_RoboschoolWalker2d-v1_0.05.sh',
 'job_RoboschoolWalker2d-v1_0.1.sh',
 'job_RoboschoolWalker2d-v1_0.2.sh',
 'job_RoboschoolWalker2d-v1_0.sh']